## Fast Python3 For Beginners

## FUNCTOOLS

### Decorator

In [1]:
def now():
    print("2017//12//18")

In [2]:
f = now
f()

2017//12//18


In [3]:
now.__name__

'now'

In [4]:
f.__name__

'now'

假设我们要增强now()函数的功能，比如，在函数调用前后自动打印日志，但又不希望修改now()函数的定义，这种<u>**在代码运行期间动态增加功能的方式**，称之为**“装饰器”**（Decorator）</u>

In [5]:
def log(func):
    def wrapper(*arg, **kw):
        print("call %s():" % func.__name__)
        return func(*arg, **kw)
    return wrapper

借助Python的@语法，把decorator置于函数的定义处：  
把**@log**放到**now()**函数的定义处，相当于执行了语句：  
*>>> now = log(now)*

In [6]:
@log
def now():
    print('2015-3-25')

In [7]:
now()

call now():
2015-3-25


由于log()是一个decorator，返回一个函数，所以，原来的now()函数仍然存在，只是现在同名的now变量指向了新的函数，于是调用now()将执行新函数，即在log()函数中返回的wrapper()函数。

In [8]:
now.__name__

'wrapper'

如果decorator本身需要传入参数，那就需要编写一个返回decorator的高阶函数，写出来会更复杂。比如，要自定义log的文本：  
和两层嵌套的decorator相比，3层嵌套的效果是这样的：  
*>>> now = log('execute')(now)*

In [9]:
def log(text):
    def decorator(func):
        def wrapper(*args, **kw):
            print('%s %s():' % (text, func.__name__))
            return func(*args, **kw)
        return wrapper
    return decorator

In [10]:
@log("execute")
def now():
    print("2017//12//8")

In [11]:
now()

execute now():
2017//12//8


functools.wraps  
为了避免有些依赖函数签名的代码执行就会出错，需要把原始函数的`__name__`等属性复制到wrapper()函数中  
只需记住在定义wrapper()的前面加上*@functools.wraps(func)*即可。

In [12]:
import functools

def log(func):
    @functools.wraps(func)
    def wrapper(*args, **kw):
        print('call %s():' % func.__name__)
        return func(*args, **kw)
    return wrapper

或者针对带参数的decorator：

In [13]:
import functools

def log(text):
    def decorator(func):
        @functools.wraps(func)
        def wrapper(*args, **kw):
            print('%s %s():' % (text, func.__name__))
            return func(*args, **kw)
        return wrapper
    return decorator

#### Practice_1

In [14]:
import time, functools

def metric(fn):
    @functools.wraps(fn)
    def wrapper(*arg, **kw):
        s_time = time.time()
        fn(*arg, **kw)
        e_time = time.time()
        print('%s executed in %s ms' % (fn.__name__, e_time - s_time))
        return fn(*arg, **kw)
    return wrapper


In [15]:
# 测试
@metric
def fast(x, y):
    time.sleep(0.0012)
    return x + y;

@metric
def slow(x, y, z):
    time.sleep(0.1234)
    return x * y * z;

f = fast(11, 22)
s = slow(11, 22, 33)
if f != 33:
    print('测试失败!')
elif s != 7986:
    print('测试失败!')

fast executed in 0.001544952392578125 ms
slow executed in 0.12355756759643555 ms


#### Practice_2

In [16]:
def log(*argg):
    def decorator(fn):
        @functools.wraps(fn)
        def wrapper(*arg, **kw):
            print("Begin call")
            print('%s executed in %s ms' % (fn.__name__, *argg))
            print("End call")
            return fn(*arg, **kw)
        return wrapper
    return decorator

In [17]:
@log
def f():
    pass

f()

TypeError: decorator() missing 1 required positional argument: 'fn'

In [18]:
@log('execute')
def f():
    pass

f()

Begin call
f executed in execute ms
End call
